In [ ]:
import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

from pylab import rcParams
rcParams['figure.figsize'] = 8, 5
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.offline import iplot

In [ ]:
df = pd.read_csv('World_University_Ranking.csv')

In [ ]:
print("Первичные строки данных:")
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
print("\nИнформация о данных:")
print(df.info())

In [ ]:
print("\nСтатистическое описание данных:")
print(df.describe())

In [ ]:
print("\nСтатистическое описание данных:")
df.describe()

In [ ]:
print("\nСтатистическое описание данных для объектных и булевых столбцов:")
df.describe(include=['object','bool'])

In [ ]:
# Количество уникальных значений для каждой колонки
for column in df.columns:
    print(f"{column}: {df[column].nunique()}")

In [ ]:
# Распределение рейтингов универ-ов по странам
plt.figure(figsize=(14, 8))
sns.countplot(y='country', data=df, order=df['country'].value_counts().index)
plt.title('Количество университетов по странам')
plt.xlabel('Количество университетов')
plt.ylabel('Страна')
plt.show()

In [ ]:
# Распределение оценок универ-ов по годам
plt.figure(figsize=(14, 8))
sns.boxplot(x='year', y='score', data=df)
plt.title('Распределение оценок университетов по годам')
plt.xlabel('Год')
plt.ylabel('Оценка')
plt.show()

In [ ]:
# Топ-10 универ-ов по качеству образования
top_education_quality = df.sort_values(by='quality_of_education', ascending=False).head(10)

plt.figure(figsize=(14, 8))

# Используем стиль для лучшего отображения
sns.set(style="whitegrid")

# Создаем палитру цветов
palette = sns.color_palette("viridis", len(top_education_quality))

# Построение графика
barplot = sns.barplot(
    x='quality_of_education', 
    y='institution', 
    data=top_education_quality, 
    palette=palette
)

# Доб заголовка и меток осей
plt.title('Топ-10 университетов по качеству образования', fontsize=16, weight='bold')
plt.xlabel('Качество образования', fontsize=14)
plt.ylabel('Университет', fontsize=14)

# Отображ знач на барах
for index, value in enumerate(top_education_quality['quality_of_education']):
    barplot.text(value, index, round(value, 2), color='black', ha="left", va="center")

# Уд верхней и правой рамок (для более эстетичного вида:))
sns.despine(left=True, bottom=True)

plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df['world_rank'], bins=50, kde=True)
plt.title('Распределение мирового рейтинга')
plt.xlabel('Мировой рейтинг')
plt.ylabel('Количество университетов')
plt.show()

In [ ]:
numeric_df = df.select_dtypes(include=[np.number])
plt.figure(figsize=(10, 8))
sns.heatmap(numeric_df.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Корреляция между различными показателями')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=df, x='year', y='score', hue='country', ci=None, estimator='mean')
plt.title('Изменение среднего балла по годам')
plt.xlabel('Год')
plt.ylabel('Средний балл')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='publications', y='score', hue='country', palette='tab10', legend=False)
plt.title('Влияние публикаций на общий балл')
plt.xlabel('Количество публикаций')
plt.ylabel('Общий балл')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='citations', y='score', hue='country', palette='tab10', legend=False)
plt.title('Влияние цитирований на общий балл')
plt.xlabel('Количество цитирований')
plt.ylabel('Общий балл')
plt.show()

In [ ]:

cols = ['world_rank', 'quality_of_education', 'alumni_employment', 'quality_of_faculty', 'publications', 'influence', 'citations', 'broad_impact', 'patents', 'score']
numeric_df = df[cols]

# Строим парные диаграммы
sns_plot = sns.pairplot(numeric_df)
sns_plot.savefig('pairplot.png')

plt.show()


In [ ]:
# Группировка данных по годам и расчет среднего балла и количества университетов
years_df = df.groupby('year')[['score']].mean().join(
    df.groupby('year')[['institution']].count()
)
years_df.columns = ['Average_Score', 'Number_of_Universities']

# Созд графиков 
trace0 = go.Scatter(
    x = years_df.index,
    y = years_df['Average_Score'],
    name = 'Average Score',
    mode = 'lines+markers'
)
trace1 = go.Scatter(
    x = years_df.index,
    y = years_df['Number_of_Universities'],
    name = 'Number of Universities',
    mode = 'lines+markers',
    yaxis='y2'
)
data = [trace0, trace1]

# Настройка
layout = go.Layout(
    title = 'University Rankings Statistics Over the Years',
    xaxis = dict(title = 'Year'),
    yaxis = dict(title = 'Average Score'),
    yaxis2 = dict(
        title = 'Number of Universities',
        overlaying = 'y',
        side = 'right'
    ),
    legend=dict(x=0.1, y=1.1)
)
fig = go.Figure(data=data, layout=layout)

iplot(fig, show_link=False)


In [ ]:

countries_df = df.groupby('country')[['score']].mean().join(
    df.groupby('country')[['institution']].count()
)
countries_df.columns = ['Average_Score', 'Number_of_Universities']

# Созд столбчатых диаграмм
trace0 = go.Bar(
    x = countries_df.index,
    y = countries_df['Average_Score'],
    name = 'Average Score'
)

trace1 = go.Bar(
    x = countries_df.index,
    y = countries_df['Number_of_Universities'],
    name = 'Number of Universities'
)

data = [trace0, trace1]

# Настройка
layout = {
    'title': 'University Rankings by Country',
    'xaxis': {'title': 'Country'},
    'yaxis': {'title': 'Values'},
    'barmode': 'group'
}
fig = go.Figure(data=data, layout=layout)

# Отображение
iplot(fig, show_link=False)


In [ ]:

cols = ['quality_of_education', 'alumni_employment', 'score']
data = df[cols]

# Созд фигуры 
fig = plt.figure(1, figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d', elev=30, azim=134)

# Очистка предыдущих графиков
plt.cla()

# Созд цветовой карты для стран
countries = df['country'].unique()
colors = plt.cm.jet(np.linspace(0, 1, len(countries)))
country_color_map = dict(zip(countries, colors))

# Отображ универ-ов в трехмерном пр-ве
for country in countries:
    subset = df[df['country'] == country]
    ax.scatter(subset['quality_of_education'], subset['alumni_employment'], subset['score'], 
               label=country, c=country_color_map[country], edgecolor='w', s=50)

# Настройка 
ax.set_xlabel('Quality of Education')
ax.set_ylabel('Alumni Employment')
ax.set_zlabel('Score')
ax.set_title('3D Visualization of University Rankings')

# Добавление легенды
ax.legend(loc='best', bbox_to_anchor=(1.05, 1), borderaxespad=0.)

plt.show()


In [ ]:
# Анализ трендов для отдельных стран
countries_of_interest = ['United States', 'United Kingdom', 'China', 'Germany', 'Japan']
plt.figure(figsize=(12, 8))

for country in countries_of_interest:
    subset = df[df['country'] == country]
    sns.lineplot(data=subset, x='year', y='score', label=country)

plt.title('Изменение среднего балла по годам для отдельных стран')
plt.xlabel('Год')
plt.ylabel('Средний балл')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()